In [29]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re
import lda
# from konlpy.tag import Okt 이거 Komoran보다 별로 복합명사 처리하기 Komoran이 유리
from konlpy.tag import Komoran
from ugtm import eGTM
import altair as alt

In [30]:
# Okt 객체를 생성합니다. (형태소 분석기)
# okt = Okt()

# Komoran 객체를 생성합니다. (형태소 분석기)
komoran = Komoran()

In [31]:
# # 복합명사 목록
compound_nouns = [
    "인공지능", "데이터베이스", "블록체인", "웨어러블", "물류센터", "무선통신", "자동화", "메타데이터", "메타버스", "작업시간", "예정시간",
    "기계학습", "강화학습", "빅데이터", "위치정보", "물류창고", "항만물류정보시스템", "항만물류", "물류관리시스템", "무인이동체", "항적정보",
    "자율주행",  "웨어러블", "정보보안", "자율운항선박", "머신러닝", "딥러닝", "디바이스", "제어창치", "야드트랙터", "로드트랙터", "항해정보",
    "자동화시스템",  "선박자동식별시스템", "무인이송차량", "자율운항", "원격제어", "제어정보", "감지센서", "온도센서", "자동화창고", "수신장치",
    "선박위치정보", "식별정보", "경로정보", "경로계획", "화물운송", "화물운송정보", "화물운송시스템", "야드크레인", "안벽크레인", "선박정보", 
    "위성정보", "위성통신", "식별장치", "운항경로", "물류정보", "화물보관", "화물차량", "경로탐색", "프레임워크", "해상운임지수", "네트워크",
    "파라미터", "콜드체인", "트레이", "다중선형회귀", "가이드레일", "연료저장탱크"
]
# 사용자 사전을 파일에 추가
with open('user_dict.txt', 'w') as f:
    for compound in compound_nouns:
        f.write(f'{compound}\tNNP\n')  # 각 단어와 태그 사이에 탭 문자를 사용
# Komoran 객체에 사용자 사전 적용
komoran = Komoran(userdic='user_dict.txt')


In [32]:
def preprocess(text):
    # 영어 단어를 따로 추출하여 보존
    english_words = re.findall(r'[a-zA-Z]+', text)
    
    # 특수 문자 제거
    text = re.sub(r'[^가-힣\s]', '', text)  # 한글과 공백을 제외하고 모두 제거
    text = re.sub(r'\s+', ' ', text)  # 중복 공백 제거
    text = text.strip()
    
    # 형태소 분석 및 명사 추출
    tokens = komoran.nouns(text)
    
    # 불용어 제거
    tokens = [word for word in tokens if word not in stop_words]
    
        # 영어 단어에서 불용어 제거
    english_words = [word for word in english_words if word.lower() not in stop_words_en]
    
    
    # 영어 단어와 한국어 명사를 결합
    tokens.extend(english_words)
    
    return ' '.join(tokens)

In [33]:
# 결과 출력 함수
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.topic_word_):
        print(f"Topic #{topic_idx}:")
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()
# def print_top_words(model, feature_names, n_top_words):
#     for topic_idx, topic in enumerate(model.topic_word_):
#         print(", ".join([f'"{feature_names[i]}"' for i in topic.argsort()[:-n_top_words - 1:-1]]))
#     print()

# # 상위 단어와 확률을 출력하는 함수
# def print_top_words(model, feature_names, n_top_words):
#     topic_word_distributions = model.topic_word_
#     for topic_idx, topic in enumerate(topic_word_distributions):
#         print(f"Topic #{topic_idx}:")
#         top_words_idx = topic.argsort()[-n_top_words:][::-1]  # 상위 n개 단어
#         for idx in top_words_idx:
#             print(f"{feature_names[idx]} ({topic[idx]:.4f})", end=" ")
#         print("\n")


In [34]:
def perform_lda_and_save(n_topics, output_file):
    # 토픽 수와 랜덤 시드 설정
    n_models = 50
    n_iter = 5000
    random_seeds = np.random.randint(0, 10000, n_models)

    # 각 토픽 수에 대해 50개의 모델을 생성하고 결과를 저장합니다.
    with open(output_file, 'w') as f:
        for seed in random_seeds:
            model = lda.LDA(n_topics=n_topics, n_iter=n_iter, random_state=seed)
            model.fit(count_matrix)
            
            f.write(f'Random Seed: {seed}\n')
            for i, topic_dist in enumerate(model.topic_word_):
                topic_words = np.array(count_vectorizer.get_feature_names_out())[np.argsort(topic_dist)][:-11:-1]
                f.write(f'Topic {i}: {" ".join(topic_words)}\n')
            f.write('\n')


In [35]:
# CSV 파일을 읽어옵니다.
file_path = '/Users/woosu/Desktop/KLIP/patent_data_v.3.4.csv'  # 파일 경로를 지정하세요.
df = pd.read_csv(file_path)
# K 열의 'sum' 열 데이터를 추출합니다.
documents = df['claim'].astype(str).tolist()


In [36]:
stop_words = [
    "물품", "장치", "결정", "방법", "단계", "사용자", "복수", "프로세서", "지지", "기상", "상황", "변화", "모드", "패킷", "서브",
    "데이터", "관리", "저장", "정보", "보안", "생성", "전자", "컴퓨터", "코드", "물류", "업체", "기자재", "상면", "기간", "표준", "발전",
    "운임", "해상", "분석", "시스템", "화물", "운송", "요청", "서버", "경우", "송사", "전압", "중심", "서비스", "모델링",
    "고상", "전송", "입력", "통신", "네트워크", "삭제", "노드", "수신", "신호", "육상", "단위", "누적", "운영", "높이", "처리",
    "메시지", "차량", "이동", "모듈", "영상", "제어", "시스템", "인식", "장비", "적재", "처리", "물질", "진단", "기기", "사항", 
    "반출", "대상", "시간", "지연", "이용", "상태", "결합", "구비", "연결", "평가", "표시", "이용한", "교차", "냉동", "신고",
    "특징", "방향", "이송", "설치", "형성", "이상", "디바이스", "물체", "할당", "실행", "확인", "단말", "중앙", "피스톤", "단일", "행동", "관리",
    "기반", "동작", "설정", "수행", "제공", "기능", "출입", "선택", "수직", "구동", "관리자", "파지", "목록", "접수", "승인", "통지",
    "프레임", "고정", "회전", "유닛", "가이드", "상부", "부재", "식별자", "이벤트", "아이템", "주문", "대응", "포인트", "표현", "오류",
    "외부", "내부", "구조", "연산", "구조물", "라인", "기초", "패키지", "과정", "하부", "본체", "수단", "구성", "몸체", "회수", 
    "해당", "결과", "프로세스", "획득", "영역", "블록", "픽셀", "사이즈", "전극", "등록", "전달", "메인", "패널", "관제소", 
    "자원", "그룹", "가시", "수식", "자루", "측위", "특성", "고객", "산출", "단말기", "의뢰", "케이슨", "파일", "조건", "취합",
    "출력", "측면", "구동", "상품", "목표", "로딩", "임무", "스테이션", "번호", "분배", "구현", "객체", "체부", "박용", "상승", "동시", "전단",
    "촬영", "항만", "운행", "본선", "선반", "앵커", "여부", "입찰", "특정", "판단", "수입", "작업", "스토커", "물리", "기구", "신청",
    "플레이트", "연장", "실제", "수집", "운용", "박스", "대역", "자산", "타겟", "형상", "연관", "워크", "완료", "각각", "비교", "상대",
    "전환", "소스", "업무", "추천", "수량", "예약", "관련", "관리부", "거래", "동부", "부가", "경사", "돌출", "삽입", "체결", "통보",
    "최대", "타입", "발생", "링크", "후보", "배치", "채널", "범위", "수송", "사용", "조합", "부착", "어부", "가동", "커버", "거더", 
    "구간", "시작", "파이프", "유지", "구획", "공간", "의약품", "탱크", "생산", "공기", "부품", "비율", "기준", "중량", "타선", "선사로",
    "샤프트", "회동", "장착", "길이", "마련", "다수", "모터", "정밀", "무게", "시설", "활성", "진단", "이력", "발주", "추출", "수평", "수용", 
    "기어", "간격", "전방", "안착", "양측", "바닥", "일단", "공급", "접촉", "세트", "반송", "최종", "상체", "진공", "로드", "차주", "송장",
    "요소", "시점", "구축", "공급", "게이트", "표준화", "구역", "크기", "속성", "규칙", "유형", "공유", "기본", "가격", "필요", "워딩", "캐리어", 
    "하우", "형태", "부의", "제작", "부분", "슬라이딩", "정도", "정지", "관제", "작업자", "인증", "대체", "화주", "클라이언트", "국제", 
    "반사", "최초", "셔틀", "관통", "포크", "벨트", "소정", "입자", "환경", "국가", "성능", "제안", "품목", "출하", "포맷", "프로비저닝", "성부", 
    "명령", "마스터", "검사", "레벨", "변환", "도어", "지시", "조절", "사이", "좌우", "상단", "제품", "상하", "구매", "수출자", "수출", "섹터",
    "하역", "송신", "응답", "이동하", "방식", "탑재", "조립", "베이스", "전후", "인터페이스", "트레이", "선사", "항목", "흡착", "점수", "선박", "오차", 
    "이하", "기판", "차원", "핸드", "함수", "제조", "용기", "엔터테인먼트", "유사", "진행", "실린더", "부상", "보조", "작동", "오버레이", "충전",
    "계산", "측정", "도메인", "허용", "동력", "방지", "노즐", "분사", "배출", "가변", "후방", "커플링", "초점", "리더", "바퀴", "러스트", "니트",
    "트롤", "핸들", "고유", "준비", "이스트", "진입", "각도", "애플리케이션", "접속", "보정", "파라미터", "보이스", "발행", "장부", "전기", "바탕",
    "안테나", "배터리", "주파수", "링크", "운전자", "개폐", "조회", "임계", "디스플레이", "추가", "프로그램", "매체", "초과", "판독", "모션",
    "테이블", "지원", "콘크리트", "상호", "일정", "개방", "대차", "외측", "그라우팅", "지대", "밀착", "중복", "순환", "다중", "통합", "레이", "융합",
    "라이", "도킹", "역기", "캐시", "임시", "트랙", "관절", "다음", "선정", "세그먼트", "반납", "스프", "차이", "실험", "업데이트", "검증", "로그", "워킹", 
    "출항", "자동", "선체", "배관", "추진", "동형", "하면", "나사", "스트림", "레인지", "용량", "패턴", "분기", "지브", "전력", "소요","가스", "별도", "마감",
    "에이전트", "리스트", "재사용", "만족", "사전", "권한", "토큰", "참여", "지점", "정보", "보관", "픽업", "설비","안내", "드럼", "감지", "수정", "대형",
    "사고", "고리", "계층", "버퍼", "무역", "지역", "지상", "물량", "식별",  "지수", "수출입", "항구", "엔진", "예상", "예정", "가상", "해외", "인양",
    "정구", "웨이트", "카운터", "조정", "캡슐", "연속", "표면", "아래", "트러스", "접근", "칼럼","직원", "데미지", "통로", "증서", "워크플로", "인접", "리치",
    "하강","내측", "베어링", "지면", "후크", "일체", "소형", "정부", "이동식", "모선","지도", "근거", "야적장",  "분포", "조명", "용도", "반응", "레이저", "메타데이터",
    "등급", "해운", "기계", "종류", "제한", "수요", "인자", "영향", "적용","트래픽", "컴퓨팅", "예방", "회피", "속력", "조종", "리스크", "속도", "평균",
    "보상", "가중치", "기사", "과거", "전용", "유도", "릴레이", "헤드", "횡단", "이어", "분리", "평행", "포트", "부하", "변수", "순위", "기록", "출발", "국내", "연계",
    "링크", "부피", "정렬", "효율", "에너지", "해양", "취약점","계약", "추진부", "밸브", "인력", "방위", "수준", "조작", "선수", "선장","비디오",
    "일부", "신한", "개수", "팔레트", "하기", "손상", "변동", "서비스", "주행", "해저", "고장", "압력", "배포", "기관", "유동", "브레이", "선로", "로프", "볼트",
    "사용자", "시스템", "센터", "인터페이스","선형","신축", "상위", "슬롯", "리퍼", "배열", "하나에", "멀티","중간", "인수", "탄성", "시공", "리드", "외면", "확장",
    "제출", "레일", "롤러", "분류", "물류창고", "감소", "제거", "초기", "성분", "검출", "분류", "전체", "정상", "기재", "거부",
]

In [37]:
stop_words_en = ["nan", "data", "dt", "nft", "os", "rsu", "db", "vhf", "mhz", "id", "lte", "gpu"] 

In [38]:
preprocessed_documents = [preprocess(doc) for doc in documents]

In [39]:
# TF-IDF 벡터화
tfidf_vectorizer = TfidfVectorizer(max_df=0.8, min_df=2,)
tfidf = tfidf_vectorizer.fit_transform(preprocessed_documents)

# TF-IDF 가중치를 기반으로 중요한 단어 선택
tfidf_feature_names = tfidf_vectorizer.get_feature_names_out()
tfidf_scores = np.array(tfidf.sum(axis=0)).flatten()
sorted_indices = np.argsort(tfidf_scores)[::-1]
# n_top_words = 2275  # 선택할 단어의 수
# top_words = sorted_indices[:n_top_words]
# selected_features = [tfidf_feature_names[i] for i in top_words]
# 임계값 설정
threshold = 0.9
selected_indices = np.where(tfidf_scores > threshold)[0]
selected_features = [tfidf_feature_names[i] for i in selected_indices]

# 선택된 단어로 새로운 Count 행렬 생성
count_vectorizer = CountVectorizer(vocabulary=selected_features)
count_matrix = count_vectorizer.fit_transform(preprocessed_documents)

# 모든 값이 0인 행 제거
nonzero_row_indices = np.where(count_matrix.sum(axis=1) != 0)[0]
count_matrix = count_matrix[nonzero_row_indices]

# 해당 문서들도 제거
preprocessed_documents = [preprocessed_documents[i] for i in nonzero_row_indices]



In [40]:
# # TF-IDF 가중치 순으로 단어와 가중치 출력
# print("TF-IDF 가중치 순으로 단어와 가중치 출력:")
# for i in sorted_indices:
#     print(f"{tfidf_feature_names[i]}: {tfidf_scores[i]:.4f}")


In [41]:
# LDA 수행 (lda 라이브러리 사용)
n_topics = 7  # 토픽의 수 (조정 가능)
model = lda.LDA(n_topics=n_topics, n_iter=5000, random_state=4170)
model.fit(count_matrix)

# for i in range(10,20):
#     n_topics = 7  # 토픽의 수 (조정 가능)
#     model = lda.LDA(n_topics=n_topics, n_iter=5000, random_state=i)
#     model.fit(count_matrix)
#     print_top_words(model, count_vectorizer.get_feature_names_out(), 10)

INFO:lda:n_documents: 401
INFO:lda:vocab_size: 412
INFO:lda:n_words: 31088
INFO:lda:n_topics: 7
INFO:lda:n_iter: 5000
INFO:lda:<0> log likelihood: -243146
INFO:lda:<10> log likelihood: -155341
INFO:lda:<20> log likelihood: -151671
INFO:lda:<30> log likelihood: -150194
INFO:lda:<40> log likelihood: -149563
INFO:lda:<50> log likelihood: -149177
INFO:lda:<60> log likelihood: -149135
INFO:lda:<70> log likelihood: -148722
INFO:lda:<80> log likelihood: -148381
INFO:lda:<90> log likelihood: -148353
INFO:lda:<100> log likelihood: -148235
INFO:lda:<110> log likelihood: -148074
INFO:lda:<120> log likelihood: -147975
INFO:lda:<130> log likelihood: -147876
INFO:lda:<140> log likelihood: -147761
INFO:lda:<150> log likelihood: -147367
INFO:lda:<160> log likelihood: -147403
INFO:lda:<170> log likelihood: -147549
INFO:lda:<180> log likelihood: -147374
INFO:lda:<190> log likelihood: -147231
INFO:lda:<200> log likelihood: -147108
INFO:lda:<210> log likelihood: -147139
INFO:lda:<220> log likelihood: -147

In [42]:
print_top_words(model, count_vectorizer.get_feature_names_out(), 10)

Topic #0:
컨테이너 위치 이미지 운반 터미널 크레인 스캐닝 야드 트레일러 핸드오버
Topic #1:
크레인 계획 자동화 터미널 정보 도착 서비스 물류 시뮬레이션 경유
Topic #2:
태그 온도 센서 스마트 창고 디지털 위치 iot qr 실시간
Topic #3:
예측 모델 블록체인 스케줄 선적 로드트랙터 항만물류 학습 알고리즘 비용
Topic #4:
위치 센서 로봇 거리 카메라 광학 운반 좌표 추적 창고
Topic #5:
원격제어 위치 충돌 위험 ais 자율운항선박 감시 추정 클라우드 송수신
Topic #6:
경로 야드트랙터 통신 목적지 무선 연료 위치 장애물 화물차량 자율주행



In [ ]:
# 각각의 토픽 수에 대해 독립적으로 실행
perform_lda_and_save(7, 'lda_topics_7.txt')


INFO:lda:n_documents: 401
INFO:lda:vocab_size: 412
INFO:lda:n_words: 31088
INFO:lda:n_topics: 7
INFO:lda:n_iter: 5000
INFO:lda:<0> log likelihood: -243146
INFO:lda:<10> log likelihood: -155132
INFO:lda:<20> log likelihood: -151298
INFO:lda:<30> log likelihood: -149722
INFO:lda:<40> log likelihood: -149113
INFO:lda:<50> log likelihood: -148521
INFO:lda:<60> log likelihood: -148141
INFO:lda:<70> log likelihood: -147236
INFO:lda:<80> log likelihood: -146583
INFO:lda:<90> log likelihood: -146284
INFO:lda:<100> log likelihood: -146041
INFO:lda:<110> log likelihood: -145788
INFO:lda:<120> log likelihood: -145569
INFO:lda:<130> log likelihood: -145318
INFO:lda:<140> log likelihood: -145262
INFO:lda:<150> log likelihood: -145357
INFO:lda:<160> log likelihood: -145048
INFO:lda:<170> log likelihood: -144992
INFO:lda:<180> log likelihood: -144554
INFO:lda:<190> log likelihood: -144597
INFO:lda:<200> log likelihood: -144760
INFO:lda:<210> log likelihood: -144822
INFO:lda:<220> log likelihood: -144

In [2]:
# perform_lda_and_save(8, 'lda_topics_8.txt')
 

In [3]:
# perform_lda_and_save(9, 'lda_topics_9.txt')

In [4]:
# # 토픽 수와 랜덤 시드 설정
# n_topics_list = [7, 8, 9]
# n_models = 50
# n_iter = 5000
# random_seeds = np.random.randint(0, 10000, n_models)

# # 결과 저장을 위한 파일명 설정
# output_files = {7: 'lda_topics_7.txt', 8: 'lda_topics_8.txt', 9: 'lda_topics_9.txt'}

# # 각 토픽 수에 대해 50개의 모델을 생성하고 결과를 저장합니다.
# for n_topics in n_topics_list:
#     with open(output_files[n_topics], 'w') as f:
#         for seed in random_seeds:
#             model = lda.LDA(n_topics=n_topics, n_iter=n_iter, random_state=seed)
#             model.fit(count_matrix)
            
#             f.write(f'Random Seed: {seed}\n')
#             for i, topic_dist in enumerate(model.topic_word_):
#                 topic_words = np.array(count_vectorizer.get_feature_names_out())[np.argsort(topic_dist)][:-11:-1]
#                 f.write(f'Topic {i}: {" ".join(topic_words)}\n')
#             f.write('\n')

In [ ]:

# 퍼플렉시티 계산 함수
def calculate_perplexity(model, dtm):
    gamma = model.transform(dtm)
    beta = model.components_ / model.components_.sum(axis=1)[:, np.newaxis]
    
    prob = np.dot(gamma, beta)
    prob[prob == 0] = np.finfo(np.float32).eps
    log_likelihood = np.sum(np.log(prob) * dtm.toarray())
    perplexity = np.exp(-log_likelihood / np.sum(dtm.toarray()))
    return perplexity

# 토픽 수에 따른 퍼플렉시티 계산
def evaluate_perplexity(dtm, min_topics=2, max_topics=10):
    results = []
    for num_topics in range(min_topics, max_topics + 1):
        model = lda.LDA(n_topics=num_topics, n_iter=5000, random_state=4)
        model.fit(dtm)
        perplexity = calculate_perplexity(model, dtm)
        results.append((num_topics, perplexity))
    return pd.DataFrame(results, columns=["num_topics", "perplexity"])

# 퍼플렉시티 평가 및 결과 출력
results = evaluate_perplexity(count_matrix, min_topics=2, max_topics=10)
print(results)

In [23]:
# 결과 시각화
chart = alt.Chart(results).mark_line(point=True).encode(
    x=alt.X('num_topics', title='Number of Topics'),
    y=alt.Y('perplexity', title='Perplexity'),
    tooltip=['num_topics', 'perplexity']
).properties(
    title='Perplexity by Number of Topics'
)

chart.show()

alt.Chart(...)

In [79]:

# 문서별 토픽 할당 추출
topic_assignments = model.doc_topic_.argmax(axis=1)

# LDA 토픽 분포 추출
topic_distributions = model.doc_topic_

# 데이터 프레임 생성
assignments_df = pd.DataFrame({'doc_id': np.arange(len(topic_assignments)), 'topic': topic_assignments})
distributions_df = pd.DataFrame(topic_distributions, columns=[f'topic_{i}' for i in range(n_topics)])
distributions_df['doc_id'] = np.arange(len(topic_distributions))
# print(topic_assignments)
# print(distributions_df['doc_id'])

In [80]:
# GTM 모델 학습 및 변환
gtm_model = eGTM(k=9, model="responsibilities").fit(topic_distributions)
gtm_means = eGTM(k=9, model="means").fit_transform(topic_distributions)
gtm_modes = eGTM(k=9, model="modes").fit_transform(topic_distributions)


In [81]:

# GTM 결과 시각화
dgtm_modes = pd.DataFrame(gtm_modes, columns=["x1", "x2"])
dgtm_modes["topic"] = topic_assignments

gtm_modes_chart = alt.Chart(dgtm_modes).mark_circle().encode(
    x='x1',
    y='x2',
    color=alt.Color('topic:N', scale=alt.Scale(scheme='viridis')),
    size=alt.value(100),
    tooltip=['x1', 'x2', 'topic:N']
).properties(title="GTM (modes)", width=400, height=400)

dgtm_means = pd.DataFrame(gtm_means, columns=["x1", "x2"])
dgtm_means["topic"] = topic_assignments

gtm_means_chart = alt.Chart(dgtm_means).mark_circle().encode(
    x='x1',
    y='x2',
    color=alt.Color('topic:N', scale=alt.Scale(scheme='viridis')),
    size=alt.value(100),
    tooltip=['x1', 'x2', 'topic:N']
).properties(title="GTM (means)", width=400, height=400)

In [82]:
# 두 시각화 결합
gtm_chart = gtm_means_chart | gtm_modes_chart
gtm_chart.display()

alt.HConcatChart(...)

In [83]:

# Construct GTM
gtm_all = eGTM(k=9, model="responsibilities").fit(topic_distributions)

gtm_modes_all = eGTM(k=9, model="modes").fit_transform(topic_distributions)

dgtm_modes_all = pd.DataFrame(gtm_modes_all, columns=["x1", "x2"])
# dgtm_modes["label"] = y

gtm_modes_all = alt.Chart(dgtm_modes_all).mark_circle().encode(
    x='x1',
    y='x2',
#    color=alt.Color('label:N',
#                    scale=alt.Scale(scheme='viridis')),
    size=alt.value(3000),
    tooltip=['x1','x2','label:N']
).properties(title = "GTM (modes)", width = 500, height = 500)





In [84]:
transformed = gtm_all.transform(topic_distributions)
gtm_all = gtm_all.optimizedModel.matY

gtm_all = pd.DataFrame(gtm_all)
# Transpose the dataframe
gtm_all_transposed = gtm_all.transpose()

# Save the transposed dataframe to CSV
gtm_all_transposed.to_csv("/Users/woosu/Desktop/KLIP/gtm_transposed_k7_19.csv")
